In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'src'))
from src.dataset_analyze import *
from src.database_op import DataBaseOP
from tqdm.notebook import tqdm
from functools import reduce
import sklearn
import matplotlib.pyplot as plt
from sklearn.datasets._samples_generator import make_blobs
from sklearn.cluster import KMeans


# 读取所有数据集。数据集较大，有3GB，请保证有足够的内存空间

In [11]:
# df = pd.read_csv('./dataset/UserBehavior.csv', header=None)
df = pd.read_csv(r'D:\Desktop\家里学习\大四上\课内\大数据课设\辅助账号/UserBehavior.csv', header=None)
df.columns = ['用户ID', '商品ID', '商品类目ID', '行为类型', '时间戳']
df

,用户ID,商品ID,商品类目ID,行为类型,时间戳
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146
...,...,...,...,...,...
100150802,999999,4797808,11120,pv,1512293403
100150803,999999,4613472,4602841,pv,1512293766
100150804,999999,3647364,2304296,pv,1512293792
100150805,999999,1903801,2304296,pv,1512293827


# 数据处理1：统计一个用户每一种行为数量 为top n的商品类目ID

In [16]:
# connect database
dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_userbehaviortopn')

# for table bigdata_userbehaviortopn, attr is fixed
attr = 'id, user_id, pv, buy, cart, fav'
top_n = 5

# maybe the statistic results is empty
def empty2zero(Series):
    if Series.empty:
        return 0
    return reduce(lambda x, y: f'{x}\t{y}', Series.items())

# for each loop we can only handle single user
# 
# for too many users, so we take user id [1, 1000)
for i in tqdm(range(1, 1000)):
    item = single_user_behavior_statistics(df, i, top_n)
    
    pv = empty2zero(item[i]['pv'])
    buy = empty2zero(item[i]['buy'])
    cart = empty2zero(item[i]['cart'])
    fav = empty2zero(item[i]['fav'])
    val = f'{i}, "{i}", "{pv}", "{buy}", "{cart}", "{fav}"'
    # insert
    dbop.insert(attr, val)
    
dbop.close()

  0%|          | 0/999 [00:00<?, ?it/s]

# 数据处理2：统计各类商品在四个行为中的排名

In [17]:
# connect database
dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_prodclass')

# for table bigdata_prodclass, attr is fixed
attr = 'id, prod_clss_id, pv, buy, cart, fav'

pcis = productsClassID_statistics(df)
for i in tqdm(range(0, 1000)):
    item = pcis[i]
    val = f'{i}, "{item[0]}", {item[1][0]}, {item[1][1]}, {item[1][2]}, {item[1][3]}'
    # insert
    dbop.insert(attr, val)
    
dbop.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# 数据处理3：统计各件商品在四个行为中的排名

In [18]:
# connect database
dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_prodid')

# for table bigdata_prodid, attr is fixed
attr = 'id, prod_id, pv, buy, cart, fav'

pis = productsID_statisctis(df)
for i in tqdm(range(0, 1000)):
    item = pis[i]
    val = f'{i}, "{item[0]}", {item[1][0]}, {item[1][1]}, {item[1][2]}, {item[1][3]}'
    # insert
    dbop.insert(attr, val)
    
dbop.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# 数据处理4：行为为buy的所有item中，排名前top_n个用户

In [19]:
# connect database
dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_buymost')

# for table bigdata_buymost, attr is fixed
attr = 'id, behavior, user_id, number'

wbm = who_buys_most(df)
for i in tqdm(range(0, 1000)):
    item = wbm[i]
    val = f'{i}, "{item[0]}", "{item[1]}", {item[2]}'
    # insert
    dbop.insert(attr, val, mode='insert')
    
dbop.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# 数据处理5：行为为cart的所有item中，排名前top_n个用户

In [22]:
# connect database
dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_cartmost')

# for table bigdata_buymost, attr is fixed
attr = 'id, behavior, user_id, number'

wbm = who_cart_most(df)
for i in tqdm(range(0, 1000)):
    item = wbm[i]
    val = f'{i}, "{item[0]}", "{item[1]}", {item[2]}'
    # insert
    dbop.insert(attr, val, mode='insert')
    
dbop.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# 数据处理6：行为为fav的所有item中，排名前top_n个用户

In [23]:
# connect database
dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_favmost')

# for table bigdata_buymost, attr is fixed
attr = 'id, behavior, user_id, number'

wbm = who_fav_most(df)
for i in tqdm(range(0, 1000)):
    item = wbm[i]
    val = f'{i}, "{item[0]}", "{item[1]}", {item[2]}'
    # insert
    dbop.insert(attr, val, mode='insert')
    
dbop.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# 机器学习应用（使用聚类算法）

In [62]:
import json

dbop = DataBaseOP(database='src/db.sqlite3')
dbop.select_table('bigdata_userbehaviortopn')
data = dbop.select()

# 取前400个用户数据(数据库中只存放了999个)
topn = 400
cnt = 0
# 簇的个数
n_clusters = 12
X = []
for tup in data:
    tmp = []
    for i in range(2, 6):
        if tup[i] == '0':
            tmp.append(0)
        else:
            tmp.append(int(tup[i].split('\t')[0].split(',')[0].split('(')[1]))

    X.append(tmp)
    cnt += 1
    if cnt > topn: 
        break
dbop.close()

y_pred = KMeans(n_clusters=n_clusters, random_state=9).fit_predict(X)


In [63]:
jsonFile = {}
jsonFile["type"] = 'force'
jsonFile['categories'] = [{'name': f'cluster{i}',
                           'keyword': {},
                           'base': f'cluster{i}Rule'} 
                          for i in range(n_clusters)]

jsonFile['nodes'] = [{'name': f'cluster{y_pred[i]}-{topn + 1}',
                     'category': float(y_pred[i])} 
                     for i in range(topn)]

clusters = [[] for _ in range(n_clusters)]
for id, label in enumerate(y_pred):
    clusters[label].append(id)
    
jsonFile['links'] = []
for cluster in clusters:
    for node_id in range(1, len(cluster)):
        jsonFile['links'].append({'source': float(cluster[node_id]), 'target': float(cluster[0])})

with open('./src/bigdata/templates/bigdata/cluster.json', 'w') as f:
    json.dump(jsonFile, f, indent=4, ensure_ascii=False)
